In [14]:
using wgregseq, CSV, DataFrames, Glob, BioSequences, CairoMakie, Statistics, StatsBase

# Set plotting style
wgregseq.plotting_style.default_makie!()

In [15]:
df_mapped = CSV.read("../../../data/barcodes/20220514_mapping/mapped_barcodes.csv", DataFrame)
first(df_mapped, 5)

,promoter
,String
1,GAAGACGACAAACTCATTCATCTCATGGCTGCCGTAGGAGACCGTGAGTGGTAATACGATAACGCACTCACCATAGATGTGTACTGTAAAGATTTCCCTGGCTTTTATGACTGGATGGAATACCCCGGTCACCCAGTTAAGTTTGTTTTTCACATACATG
2,GAGGACGACATACGCATTCATCTCTTGGCTGCCGTACGAGACTGTGAGTGGTCAGACGATAATGCCCTCACCATAAATGTGCAGTTTCATGATTTCCCTGTATTAGATGACTGGATGGATTGCCCTTATCACCCGATTAACTGCGTTTTTCACATACTTG
3,GATTACGACAAACTCATTCATCTCATGGCTGCCGTACGAGACTGTGAGTCGTCATGCGATAACGCACTCACCATAGATTTGTAGTCTAATGATTTCCCTGGATTTTATGACAGGATGGCTTTCCCTGCTCCCCCGACTAAGTTTGCTTTTCACATACTTG
4,GATAACGACTAACTCATTCATCTCATGGCTGCCGTACGAGACTTTTCCTGGGCAGAAGATAACCCACTCACCATAAATGAGCAGTTTAATGATTTCCCTGGATGTTGTGACTGTATGGATTACCCTGATCACCCGGTTAAATTTGTTATTCCCATACTTG
5,GATGACGATAAATTCATTCATCTGATGGCTGCCGCACGAGACTCTGAGTGGTCAGACGATAATGCACTCACCATAAATGTACAGCTAAATGTTTTCCCTCGATTTTATCACTGGATGGATTCCCCTGATCACCGGGTTCAGTTTGTTTTTCACATCCTTG


In [16]:
# Find non-unique entries
index = df_mapped[!, ["barcode"]] |> nonunique

# Remove non-unique entries
df_mapped_un = df_mapped[ .~ index, :]

first(df_mapped_un, 5)

,promoter
,String
1,GAAGACGACAAACTCATTCATCTCATGGCTGCCGTAGGAGACCGTGAGTGGTAATACGATAACGCACTCACCATAGATGTGTACTGTAAAGATTTCCCTGGCTTTTATGACTGGATGGAATACCCCGGTCACCCAGTTAAGTTTGTTTTTCACATACATG
2,GAGGACGACATACGCATTCATCTCTTGGCTGCCGTACGAGACTGTGAGTGGTCAGACGATAATGCCCTCACCATAAATGTGCAGTTTCATGATTTCCCTGTATTAGATGACTGGATGGATTGCCCTTATCACCCGATTAACTGCGTTTTTCACATACTTG
3,GATTACGACAAACTCATTCATCTCATGGCTGCCGTACGAGACTGTGAGTCGTCATGCGATAACGCACTCACCATAGATTTGTAGTCTAATGATTTCCCTGGATTTTATGACAGGATGGCTTTCCCTGCTCCCCCGACTAAGTTTGCTTTTCACATACTTG
4,GATAACGACTAACTCATTCATCTCATGGCTGCCGTACGAGACTTTTCCTGGGCAGAAGATAACCCACTCACCATAAATGAGCAGTTTAATGATTTCCCTGGATGTTGTGACTGTATGGATTACCCTGATCACCCGGTTAAATTTGTTATTCCCATACTTG
5,GATGACGATAAATTCATTCATCTGATGGCTGCCGCACGAGACTCTGAGTGGTCAGACGATAATGCACTCACCATAAATGTACAGCTAAATGTTTTCCCTCGATTTTATCACTGGATGGATTCCCCTGATCACCGGGTTCAGTTTGTTTTTCACATCCTTG


In [ ]:
df_all = DataFrame()
for gc in ["Etoh", "Gluc", "LB", "XyAr"]
    df_cdna = CSV.read(
        "../../../data/processed_barcodes/20221114_barcode/$(gc)_cDNA_collapsed.txt", 
        DataFrame, 
        ignorerepeated=true, 
        delim=" ",
        header=["counts", "barcode"]
        )


    df_gdna = CSV.read(
        "../../../data/processed_barcodes/20221114_barcode/$(gc)_gDNA_collapsed.txt", 
        DataFrame, 
        ignorerepeated=true, 
        delim=" ",
        header=["counts", "barcode"]
        )
        

    dfg = leftjoin(df_gdna, df_mapped_un, on="barcode", makeunique=true)
    dfc = leftjoin(df_cdna, df_mapped_un, on="barcode", makeunique=true)
    _df = outerjoin(dfg, dfc, on=["barcode"], renamecols="_DNA" =>"_RNA")
  
    _df.counts_RNA = coalesce.(_df.counts_RNA, 0)
    _df.counts_DNA = coalesce.(_df.counts_DNA, 0)
    
    insertcols!(_df, 1, :growth_condition => gc)
    insertcols!(_df, 1, :name => wgregseq.utils.joincols(_df, :name_RNA, :name_DNA))
    insertcols!(_df, 1, :promoter => wgregseq.utils.joincols(_df, :promoter_RNA, :promoter_DNA))

    _df = _df[:, [:growth_condition, :name, :counts_DNA, :counts_RNA, :promoter, :barcode]]
    println("$gc done")
    CSV.write("../../../data/processed_barcodes/20221114_barcode/$(gc)_joined.txt", _df)
end

In [19]:
df = CSV.read("../../../data/processed_barcodes/20221114_barcode/XyAr_joined.txt", DataFrame)

,growth_condition,name,counts_DNA,counts_RNA,promoter
,String7,String?,Int64,Int64,String?
1,XyAr,TSS_1414_storz_regulondb,587,4,GAAGACGACAAACTCATTCATCTCATGGCTGCCGTAGGAGACCGTGAGTGGTAATACGATAACGCACTCACCATAGATGTGTACTGTAAAGATTTCCCTGGCTTTTATGACTGGATGGAATACCCCGGTCACCCAGTTAAGTTTGTTTTTCACATACATG
2,XyAr,TSS_1414_storz_regulondb,648,4,GATTACGACAAACTCATTCATCTCATGGCTGCCGTACGAGACTGTGAGTCGTCATGCGATAACGCACTCACCATAGATTTGTAGTCTAATGATTTCCCTGGATTTTATGACAGGATGGCTTTCCCTGCTCCCCCGACTAAGTTTGCTTTTCACATACTTG
3,XyAr,TSS_1414_storz_regulondb,116,3,GATAACGACTAACTCATTCATCTCATGGCTGCCGTACGAGACTTTTCCTGGGCAGAAGATAACCCACTCACCATAAATGAGCAGTTTAATGATTTCCCTGGATGTTGTGACTGTATGGATTACCCTGATCACCCGGTTAAATTTGTTATTCCCATACTTG
4,XyAr,TSS_1414_storz_regulondb,1713,19,GATGACGATAAATTCATTCATCTGATGGCTGCCGCACGAGACTCTGAGTGGTCAGACGATAATGCACTCACCATAAATGTACAGCTAAATGTTTTCCCTCGATTTTATCACTGGATGGATTCCCCTGATCACCGGGTTCAGTTTGTTTTTCACATCCTTG
5,XyAr,TSS_1414_storz_regulondb,1099,18,GATGACGACAAAATCGTTCACCTTATGCCTGCCGTAAGAGTCTGTGAGTGGTCAGACGATAACGGACTCACCTTAAATCTGCAGCTTAAGAATTTCCCTGGATTTTATGACTGGATGGATTAGCCTGATCACCCGGTTAAGTTTTATTTTCACATACTTG
6,XyAr,TSS_1414_storz_regulondb,3696,47,GATGACGAGAAACTCACTATTCTCATGGCTGCCGTACGAGACTGTGAGTGGTCAGACGATAACGCACTCTCCAAAAATGTGAAGTTCAATGATTTCTCTGGATTTTATGACTGGATCGTTTACCCTTATCACCCCGTTATGTTTGGTTTTTACATACTTG
7,XyAr,TSS_1414_storz_regulondb,5578,256,GATGACCACAGACTCAGTCATCTCATGGCTGCCGTACGAGCCTGTGCGTGGTCAGACGATAACGAACTCACCATAAATGTGCAGTTTAAAGAATTCCCTGGATACTATGACTTGCTGGATTTCCCTGATCACCCGGCTACGTTTGTTTTTCACATAATTG
8,XyAr,TSS_1414_storz_regulondb,499,8,GATGACGACAAACTCATTCATCTCATGGCTGCCTGCCGACACTGTGAGTGGTCAGACGATAACGGCCTCGCCATAAATATGCACTTTCATCATTTCCCTGGATTTTATGACTGGATGGCTTACCCTGATCACCCGATTAAGCTTGTTTTTCAGACACTTG
9,XyAr,TSS_1414_storz_regulondb,272,7,GATGACCACAAACTCATTCGTCTCATGGGTACCGTACGTGACTGTGAGTGGTCAGACGATAACGCACTCACCATAAATCGGCAGTTTAATGGATTCCCTGCATTTTGTGACTTGATGGATTACCCCGATCATCCGGTGTAGTTTGTTTTTCATATACTTG


In [29]:
df_filt = df[.~ ismissing.(df.name), :]

,growth_condition,name,counts_DNA,counts_RNA,promoter
,String7,String?,Int64,Int64,String?
1,XyAr,TSS_1414_storz_regulondb,587,4,GAAGACGACAAACTCATTCATCTCATGGCTGCCGTAGGAGACCGTGAGTGGTAATACGATAACGCACTCACCATAGATGTGTACTGTAAAGATTTCCCTGGCTTTTATGACTGGATGGAATACCCCGGTCACCCAGTTAAGTTTGTTTTTCACATACATG
2,XyAr,TSS_1414_storz_regulondb,648,4,GATTACGACAAACTCATTCATCTCATGGCTGCCGTACGAGACTGTGAGTCGTCATGCGATAACGCACTCACCATAGATTTGTAGTCTAATGATTTCCCTGGATTTTATGACAGGATGGCTTTCCCTGCTCCCCCGACTAAGTTTGCTTTTCACATACTTG
3,XyAr,TSS_1414_storz_regulondb,116,3,GATAACGACTAACTCATTCATCTCATGGCTGCCGTACGAGACTTTTCCTGGGCAGAAGATAACCCACTCACCATAAATGAGCAGTTTAATGATTTCCCTGGATGTTGTGACTGTATGGATTACCCTGATCACCCGGTTAAATTTGTTATTCCCATACTTG
4,XyAr,TSS_1414_storz_regulondb,1713,19,GATGACGATAAATTCATTCATCTGATGGCTGCCGCACGAGACTCTGAGTGGTCAGACGATAATGCACTCACCATAAATGTACAGCTAAATGTTTTCCCTCGATTTTATCACTGGATGGATTCCCCTGATCACCGGGTTCAGTTTGTTTTTCACATCCTTG
5,XyAr,TSS_1414_storz_regulondb,1099,18,GATGACGACAAAATCGTTCACCTTATGCCTGCCGTAAGAGTCTGTGAGTGGTCAGACGATAACGGACTCACCTTAAATCTGCAGCTTAAGAATTTCCCTGGATTTTATGACTGGATGGATTAGCCTGATCACCCGGTTAAGTTTTATTTTCACATACTTG
6,XyAr,TSS_1414_storz_regulondb,3696,47,GATGACGAGAAACTCACTATTCTCATGGCTGCCGTACGAGACTGTGAGTGGTCAGACGATAACGCACTCTCCAAAAATGTGAAGTTCAATGATTTCTCTGGATTTTATGACTGGATCGTTTACCCTTATCACCCCGTTATGTTTGGTTTTTACATACTTG
7,XyAr,TSS_1414_storz_regulondb,5578,256,GATGACCACAGACTCAGTCATCTCATGGCTGCCGTACGAGCCTGTGCGTGGTCAGACGATAACGAACTCACCATAAATGTGCAGTTTAAAGAATTCCCTGGATACTATGACTTGCTGGATTTCCCTGATCACCCGGCTACGTTTGTTTTTCACATAATTG
8,XyAr,TSS_1414_storz_regulondb,499,8,GATGACGACAAACTCATTCATCTCATGGCTGCCTGCCGACACTGTGAGTGGTCAGACGATAACGGCCTCGCCATAAATATGCACTTTCATCATTTCCCTGGATTTTATGACTGGATGGCTTACCCTGATCACCCGATTAAGCTTGTTTTTCAGACACTTG
9,XyAr,TSS_1414_storz_regulondb,272,7,GATGACCACAAACTCATTCGTCTCATGGGTACCGTACGTGACTGTGAGTGGTCAGACGATAACGCACTCACCATAAATCGGCAGTTTAATGGATTCCCTGCATTTTGTGACTTGATGGATTACCCCGATCATCCGGTGTAGTTTGTTTTTCATATACTTG


In [31]:
df_filt[(df_filt.counts_DNA .> 0) .& (df_filt.counts_RNA .> 0), :]

,growth_condition,name,counts_DNA,counts_RNA,promoter
,String7,String?,Int64,Int64,String?
1,XyAr,TSS_1414_storz_regulondb,587,4,GAAGACGACAAACTCATTCATCTCATGGCTGCCGTAGGAGACCGTGAGTGGTAATACGATAACGCACTCACCATAGATGTGTACTGTAAAGATTTCCCTGGCTTTTATGACTGGATGGAATACCCCGGTCACCCAGTTAAGTTTGTTTTTCACATACATG
2,XyAr,TSS_1414_storz_regulondb,648,4,GATTACGACAAACTCATTCATCTCATGGCTGCCGTACGAGACTGTGAGTCGTCATGCGATAACGCACTCACCATAGATTTGTAGTCTAATGATTTCCCTGGATTTTATGACAGGATGGCTTTCCCTGCTCCCCCGACTAAGTTTGCTTTTCACATACTTG
3,XyAr,TSS_1414_storz_regulondb,116,3,GATAACGACTAACTCATTCATCTCATGGCTGCCGTACGAGACTTTTCCTGGGCAGAAGATAACCCACTCACCATAAATGAGCAGTTTAATGATTTCCCTGGATGTTGTGACTGTATGGATTACCCTGATCACCCGGTTAAATTTGTTATTCCCATACTTG
4,XyAr,TSS_1414_storz_regulondb,1713,19,GATGACGATAAATTCATTCATCTGATGGCTGCCGCACGAGACTCTGAGTGGTCAGACGATAATGCACTCACCATAAATGTACAGCTAAATGTTTTCCCTCGATTTTATCACTGGATGGATTCCCCTGATCACCGGGTTCAGTTTGTTTTTCACATCCTTG
5,XyAr,TSS_1414_storz_regulondb,1099,18,GATGACGACAAAATCGTTCACCTTATGCCTGCCGTAAGAGTCTGTGAGTGGTCAGACGATAACGGACTCACCTTAAATCTGCAGCTTAAGAATTTCCCTGGATTTTATGACTGGATGGATTAGCCTGATCACCCGGTTAAGTTTTATTTTCACATACTTG
6,XyAr,TSS_1414_storz_regulondb,3696,47,GATGACGAGAAACTCACTATTCTCATGGCTGCCGTACGAGACTGTGAGTGGTCAGACGATAACGCACTCTCCAAAAATGTGAAGTTCAATGATTTCTCTGGATTTTATGACTGGATCGTTTACCCTTATCACCCCGTTATGTTTGGTTTTTACATACTTG
7,XyAr,TSS_1414_storz_regulondb,5578,256,GATGACCACAGACTCAGTCATCTCATGGCTGCCGTACGAGCCTGTGCGTGGTCAGACGATAACGAACTCACCATAAATGTGCAGTTTAAAGAATTCCCTGGATACTATGACTTGCTGGATTTCCCTGATCACCCGGCTACGTTTGTTTTTCACATAATTG
8,XyAr,TSS_1414_storz_regulondb,499,8,GATGACGACAAACTCATTCATCTCATGGCTGCCTGCCGACACTGTGAGTGGTCAGACGATAACGGCCTCGCCATAAATATGCACTTTCATCATTTCCCTGGATTTTATGACTGGATGGCTTACCCTGATCACCCGATTAAGCTTGTTTTTCAGACACTTG
9,XyAr,TSS_1414_storz_regulondb,272,7,GATGACCACAAACTCATTCGTCTCATGGGTACCGTACGTGACTGTGAGTGGTCAGACGATAACGCACTCACCATAAATCGGCAGTTTAATGGATTCCCTGCATTTTGTGACTTGATGGATTACCCCGATCATCCGGTGTAGTTTGTTTTTCATATACTTG
